# Screaming Frog 404 Links Summary

@ Author: Andrea Moro<br>
@ Date: 2020-04-10
@ Version: 0.1

### Description
This snippet has been put together to facilitate the reporting on the broken links a website may contain. 

The data source is assumed to be an **"All in link"** report from Screaming Frog.

In [1]:
import pandas as pd
file="/Users/andreamoro/Downloads/de_all_inlinks.xlsx"

In [2]:
tab = 0  # Here you can replace with an exact name you know it
dt = pd.read_excel(file, sheet_name=tab)

In [3]:
# Get only the 404 pages
dt = dt[dt['Status Code'].isin([404,410])].iloc[:,[1, 2]]
dt.columns = ['Source','Destination'] 

In [4]:
dt.head(10)

,Source,Destination
0,https://www.test.com/index.html?id=help&file=f...,https://www.test.com/index.html?id=help&file=o...
1,https://www.test.com/index.html?id=help&file=o...,https://www.test.com/index.html?id=help&file=o...
2,https://www.test.com/index.html?id=help&file=p...,https://www.test.com/index.html?id=help&file=o...
3,https://www.test.com/index.html?id=help&file=p...,https://www.test.com/index.html?id=help&file=o...
4,https://www.test.com/index.html?id=help&file=m...,https://www.test.com/index.html?id=help&file=o...
5,https://www.test.com/index.html?id=help&file=w...,https://www.test.com/index.html?id=help&file=o...
6,https://www.test.com/index.html?id=help&file=p...,https://www.test.com/index.html?id=help&file=o...
7,https://www.test.com/index.html?id=help&file=r...,https://www.test.com/index.html?id=help&file=o...
8,https://www.test.com/index.html?id=help&file=q...,https://www.test.com/index.html?id=help&file=o...
9,https://www.test.com/index.html?id=help&file=d...,https://www.test.com/index.html?id=help&file=o...


In [5]:
# import pandas_profiling
# pandas_profiling.ProfileReport(dt)

In [6]:
#dt = dt.loc[:, ~dt.columns.str.contains('Source_Count', case=False)]

In [7]:
# Calculate how many time a given source & destination have been used
dt = dt.merge(dt.Source.value_counts().reset_index().rename(columns={"index": "Source", "Source": "Source_Count"}), on='Source')
dt = dt.merge(dt.Destination.value_counts().reset_index().rename(columns={"index": "Destination", "Destination": "Destination_Count"}), on='Destination')

In [8]:
# Adjust the offset below for your Sitewide consideration
sitewide_offset = 2

# Aggregate some data
source_count = dt.Source.value_counts().sum()
broken_links_total_source = dt.Source.nunique()
site_wide_broken_links_pages = len(dt.Source[dt.Source_Count>sitewide_offset].unique())
one_broken_link_pages = broken_links_total_source - site_wide_broken_links_pages 

print("{:<42}{:>10,}\n\n{:<42}{:>10,}\n{:<42}{:>10,}\n{:<42}{:>10,}".format('# Source URLs containing Broken Links:', source_count,
                                                                            '# Unique URLs:', broken_links_total_source,
                                                                            '|__ With sitewide broken links:', site_wide_broken_links_pages,
                                                                            '|__ With a unique broken link:', one_broken_link_pages))

# Gather the # of self-referenced Source URLs resolving in a broken link
#
# Here I'm making use of the newest functionality of Python
# the Assignment Operator (or Walrus operator)
# https://www.python.org/dev/peps/pep-0572/
uniq_tot_urls = 1
tot_links_from_uniq = 0
[(uniq_tot_urls := uniq_tot_urls + x[0], tot_links_from_uniq := tot_links_from_uniq + x[1]) 
 for x in enumerate(dt.Destination_Count[(dt.Source_Count == 1) & (dt.Destination_Count > 1) & (dt.Source == dt.Destination)])]

if tot_links_from_uniq > 0:
    plural = "s" if tot_links_from_uniq > 1 else ""
    print(f'    |__ {uniq_tot_urls} URL{plural} generate{plural} {tot_links_from_uniq} self-referenced link{plural}')
    print(f'        They may not necessarily be unique.')
else:
    print(f'    |___ No self-referenced sitewide links found')
    
broken_urls_total = dt.Destination.nunique() 
BrokenLinksSiteWide = len(dt.Destination[dt.Destination_Count>sitewide_offset].unique())
BrokenLinksSiteWideSum = dt.Destination_Count[dt.index.isin(dt.Destination[dt.Destination_Count>sitewide_offset].drop_duplicates().index)].sum()
BrokenLinksSingle = len(dt.Destination[dt.Destination_Count==sitewide_offset].unique())\
                        + (dt.Destination_Count[dt.Destination_Count==1].sum() if sitewide_offset > 1 else 0)
BrokenLinksSingleSum = dt.Destination_Count[dt.index.isin(dt.Destination[dt.Destination_Count==sitewide_offset].drop_duplicates().index)].sum()\
                        + (dt.Destination_Count[dt.Destination_Count==1].sum() if sitewide_offset > 1 else 0)
#dt.Destination_Count[dt.Destination_Count==1].sum()

plural = "s" if sitewide_offset > 1 else ""
single_offset_str = f"up to {sitewide_offset} Link{plural}"
print(f"\n\nSitewide offset: {sitewide_offset:>35}")
print("\n{:>52}{:>10}".format('# of URLs', '  (In # pages)'))
print("{:<42}{:>10,}\n{:<42}{:>10,}  ({:>10,})".format('Total Unique Broken URLs:', broken_urls_total,
                                                                            '|__ Including Sitewide Links:', BrokenLinksSiteWide, BrokenLinksSiteWideSum))
print("{:<42}{:>10,}  ({:>10,})".format(f'|__ Including {single_offset_str}:', BrokenLinksSingle, BrokenLinksSingleSum))
      
                                                                            

# Source URLs containing Broken Links:        15,361

# Unique URLs:                                   633
|__ With sitewide broken links:                  630
|__ With a unique broken link:                     3
    |__ 2 URLs generates 1262 self-referenced links
        They may not necessarily be unique.


Sitewide offset:                                   2

                                           # of URLs  (In # pages)
Total Unique Broken URLs:                        686
|__ Including Sitewide Links:                    157  (    14,669)
|__ Including up to 2 Links:                     529  (       692)


### Some example of data subsetting

In [9]:
# Links where the Source matches the Destination
dt[dt.Source == dt.Destination]

,Source,Destination,Source_Count,Destination_Count
14,https://www.test.com/index.html?id=help&file=o...,https://www.test.com/index.html?id=help&file=o...,40,30
87,https://www.test.com/index.html?id=services,https://www.test.com/index.html?id=services,32,650
88,https://www.test.com/index.html?id=services,https://www.test.com/index.html?id=services,32,650
692,https://www.test.com/index.html?id=help&file=c...,https://www.test.com/index.html?id=help&file=c...,40,38
768,https://www.test.com/index.html?id=aboutRS/ema...,https://www.test.com/index.html?id=aboutRS/ema...,29,649
...,...,...,...,...
15354,https://www.test.com/index.html?id=schneider_h...,https://www.test.com/index.html?id=schneider_h...,17,3
15355,https://www.test.com/index.html?id=schneider_h...,https://www.test.com/index.html?id=schneider_h...,17,3
15357,https://www.test.com/index.html?id=eaton-easy,https://www.test.com/index.html?id=eaton-easy,16,2
15359,https://www.test.com/index.html?id=schneider-t...,https://www.test.com/index.html?id=schneider-t...,17,3


In [10]:
# Links where the Destination is greater or equal to the specified offset
dt[dt.Destination_Count>=sitewide_offset]

,Source,Destination,Source_Count,Destination_Count
0,https://www.test.com/index.html?id=help&file=f...,https://www.test.com/index.html?id=help&file=o...,40,30
1,https://www.test.com/index.html?id=help&file=o...,https://www.test.com/index.html?id=help&file=o...,40,30
2,https://www.test.com/index.html?id=help&file=p...,https://www.test.com/index.html?id=help&file=o...,40,30
3,https://www.test.com/index.html?id=help&file=p...,https://www.test.com/index.html?id=help&file=o...,42,30
4,https://www.test.com/index.html?id=help&file=m...,https://www.test.com/index.html?id=help&file=o...,40,30
...,...,...,...,...
15356,https://www.test.com/index.html?id=eaton-easy&...,https://www.test.com/index.html?id=eaton-easy,16,2
15357,https://www.test.com/index.html?id=eaton-easy,https://www.test.com/index.html?id=eaton-easy,16,2
15358,https://www.test.com/index.html?id=schneider-t...,https://www.test.com/index.html?id=schneider-t...,17,3
15359,https://www.test.com/index.html?id=schneider-t...,https://www.test.com/index.html?id=schneider-t...,17,3


In [11]:
# To get some insights, simply add the .to_clipboard() at the end and you will get a ready to paste into Excel set of data.